In [18]:
import numpy as np
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

In [20]:
df = pd.read_csv('NHIS_OPEN_GJ_2017_100.csv',engine = 'python')

In [21]:
df_sample = df[['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤','성별코드', '연령대코드(5세단위)','수축기혈압','이완기혈압','식전혈당(공복혈당)','신장(5Cm단위)', '체중(5Kg단위)', '허리둘레']]


In [22]:
df_sample['총콜레스테롤']

0         193.0
1         228.0
2         136.0
3         201.0
4         199.0
          ...  
999995    198.0
999996    133.0
999997    205.0
999998    122.0
999999    225.0
Name: 총콜레스테롤, Length: 1000000, dtype: float64

In [23]:
df_sample = df_sample.dropna()

In [24]:
df_sample = df_sample.loc[(df_sample['허리둘레'] >= 40) & (df_sample['허리둘레'] <= 140)]

In [25]:
def bmi(height,weight):
    m = height / 100
    return weight / (m ** 2)
def diabetes(x):
    if x >= 100 and x < 126:
        return "전단계"
    elif x >= 127:
        return "당뇨"
    else:
        return "정상"

def fat(bmi) :
    if(bmi < 25):
        return '정상'
    elif(bmi >= 25 and bmi < 30):
        return '과체중'
    else:
        return '비만'

def pulse(low,high):
    if low >= 100 or high >= 160:
        return "2기고혈압"
    elif 90 <= low < 100 or 140 <= high < 160:
        return "1기"
    elif 80 <= low < 90 or 120 <= high < 140:
        return "전단계"
    else:
        return "정상"

def bokbu(gen, hurry):
    if gen == 1 and hurry >= 90:
        return "비만"
    elif gen == 2 and hurry >= 85:
        return "비만"
    else:
        return "정상"

def colesterol(all, ldl, hdl):
    if 139 <= all <200 or 130<= ldl <= 159 or 35 <= hdl <=  59:
        return 1
    elif all <= 200 or ldl <= 130 or hdl <= 60:
        return 0
    else:
        return -1

In [26]:
df_sample['당뇨병'] = df_sample['식전혈당(공복혈당)'].map(diabetes)
df_sample['고혈압'] = np.vectorize(pulse)(df_sample['이완기혈압'], df_sample['수축기혈압'])
df_sample['bmi'] = np.vectorize(bmi)(df_sample['신장(5Cm단위)'], df_sample['체중(5Kg단위)'])
df_sample['비만도'] = df_sample['bmi'].map(fat)
df_sample['복부비만'] = np.vectorize(bokbu)(df_sample['성별코드'], df_sample['허리둘레'])
df_sample['고지혈증'] = np.vectorize(colesterol)(df_sample['총콜레스테롤'],df_sample['LDL콜레스테롤'],df_sample['HDL콜레스테롤'])

In [27]:
df_sample['당뇨병'].value_counts()

정상     626457
전단계    296442
당뇨      73840
Name: 당뇨병, dtype: int64

In [28]:
df_sample['고지혈증'].value_counts()


 1    863284
 0    104902
-1     28553
Name: 고지혈증, dtype: int64

In [29]:
df_sample.describe()

,총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,성별코드,연령대코드(5세단위),수축기혈압,이완기혈압,식전혈당(공복혈당),신장(5Cm단위),체중(5Kg단위),허리둘레,bmi,고지혈증
count,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000,996739.000000
mean,195.501086,132.077642,56.930715,112.999331,1.469835,10.528189,122.425408,76.044000,100.433773,162.223461,63.258004,81.172262,23.912151,0.837462
std,38.669277,102.201252,17.229834,35.842479,0.499090,2.841360,14.553429,9.891631,24.204515,9.284908,12.516185,9.594750,3.516292,0.439787
min,30.000000,1.000000,1.000000,1.000000,1.000000,5.000000,58.000000,30.000000,25.000000,130.000000,25.000000,40.000000,12.486993,-1.000000
25%,169.000000,73.000000,46.000000,89.000000,1.000000,8.000000,112.000000,70.000000,88.000000,155.000000,55.000000,74.000000,21.484375,1.000000
50%,193.000000,106.000000,55.000000,111.000000,1.000000,10.000000,120.000000,76.000000,96.000000,160.000000,60.000000,81.000000,23.875115,1.000000
75%,219.000000,159.000000,66.000000,135.000000,2.000000,13.000000,131.000000,82.000000,105.000000,170.000000,70.000000,87.700000,25.951557,1.000000
max,2344.000000,9490.000000,8110.000000,5119.000000,2.000000,18.000000,273.000000,185.000000,852.000000,190.000000,140.000000,140.000000,45.714286,1.000000


In [32]:

def Entropy(df, attribute) :
    list1  = list(df[attribute].unique())
    etp = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        etp -=  q1*np.log2(q1)
    return etp

def Group_info(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_Entropy = Entropy(df,fix_attribute)
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Entropy(q2,fix_attribute) 
    
    data = info_Entropy - info_data
    
    return data

info = np.array([("성별코드"),("고혈압"),("비만도"),("복부비만"),("연령대코드(5세단위)"),("고지혈증"),("총콜레스테롤"),("HDL콜레스테롤"),("LDL콜레스테롤")])

for i in info :
    print(i,":",Group_info(df_sample, i,"당뇨병") * 1000)

성별코드 : 13.76310992524532
고혈압 : 29.599823180006535
비만도 : 20.71876197429301
복부비만 : 20.92423466753779
연령대코드(5세단위) : 46.91403754450207
고지혈증 : 0.35700174649733185
총콜레스테롤 : 12.97783939851982
HDL콜레스테롤 : 15.825036610049414
LDL콜레스테롤 : 12.470371371869238


In [10]:
#지니 지수 https://lucy-the-marketer.kr/ko/growth/decision-tree-and-impurity/

In [11]:
def Gini(df, attribute) :
    list1  = list(df[attribute].unique())
    gini = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        gini += (q1**2)
    return 1 - gini

def Group_Gini(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Gini(q2,fix_attribute) 
    
    return info_data

info = np.array([("성별코드"),("고혈압"),("비만도"),("복부비만"),("연령대코드(5세단위)")])

for i in info :
    print(i,":",Group_Gini(df_sample, i,"당뇨병"))

성별코드 : 0.5132477500810447
고혈압 : 0.5067788048087857
비만도 : 0.5102727536675177
복부비만 : 0.5110472000612207
연령대코드(5세단위) : 0.501566998920739


In [12]:
df_sample[["성별코드","고혈압","비만도","복부비만","연령대코드(5세단위)"]].describe()

,성별코드,연령대코드(5세단위)
count,993643.000000,993643.000000
mean,1.463026,10.569117
std,0.498631,2.861437
min,1.000000,5.000000
25%,1.000000,8.000000
50%,1.000000,11.000000
75%,2.000000,13.000000
max,2.000000,18.000000


In [15]:
list(df.columns)

['기준년도',
 '가입자일련번호',
 '성별코드',
 '연령대코드(5세단위)',
 '시도코드',
 '신장(5Cm단위)',
 '체중(5Kg단위)',
 '허리둘레',
 '시력(좌)',
 '시력(우)',
 '청력(좌)',
 '청력(우)',
 '수축기혈압',
 '이완기혈압',
 '식전혈당(공복혈당)',
 '총콜레스테롤',
 '트리글리세라이드',
 'HDL콜레스테롤',
 'LDL콜레스테롤',
 '혈색소',
 '요단백',
 '혈청크레아티닌',
 '(혈청지오티)AST',
 '(혈청지오티)ALT',
 '감마지티피',
 '흡연상태',
 '음주여부',
 '구강검진수검여부',
 '치아우식증유무',
 '결손치유무',
 '치아마모증유무',
 '제3대구치(사랑니)이상',
 '치석',
 '데이터공개일자']